### Student Information
Name: Kevin Tuan

Student ID: 107065527

GitHub ID: kevin810323

Kaggle name: Kevin Tuan

Kaggle private scoreboard snapshot:

[Snapshot](img/pic0.png)

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the [DM19-Lab2-Master Repo](https://github.com/EvaArevalo/DM19-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/t/179d01d4dd984fc5ac45a894822479dd) regarding Emotion Recognition on Twitter. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the score (ie. 20% of 30% )

    - **Top 41% - 100%**: Get (101-x)% of the score, where x is your ranking in the leaderboard (ie. (101-x)% of 30% )   
    Submit your last submission __BEFORE the deadline (Nov. 23rd 11:59 pm, Saturday)__. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**.


You can submit your homework following these guidelines: [Git Intro & How to hand your homework](https://github.com/EvaArevalo/DM19-Lab1-Master/blob/master/Git%20Intro%20%26%20How%20to%20hand%20your%20homework.ipynb), but make sure to fork the [DM19-Lab2-Homework](https://github.com/EvaArevalo/DM19-Lab2-Homework) repository this time! Also please __DON´T UPLOAD HUGE DOCUMENTS__, please use Git ignore for that.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 26th 11:59 pm, Tuesday)__. 

In [2]:
### Begin Assignment Here

In [314]:
import pandas as pd
import numpy as np
import json
import re
import scipy.sparse
import os


In [17]:
with open('tweets_DM.json') as f:
    data = [json.loads(line) for line in f]


In [255]:
data[0]

{'_score': 391,
 '_index': 'hashtag_tweets',
 '_source': {'tweet': {'hashtags': ['Snapchat'],
   'tweet_id': '0x376b20',
   'text': 'People who post "add me on #Snapchat" must be dehydrated. Cuz man.... that\'s <LH>'}},
 '_crawldate': '2015-05-23 11:42:47',
 '_type': 'tweets'}

In [256]:
tw = [ i['_source']['tweet'] for i in data ]

In [257]:
tw[0]

{'hashtags': ['Snapchat'],
 'tweet_id': '0x376b20',
 'text': 'People who post "add me on #Snapchat" must be dehydrated. Cuz man.... that\'s <LH>'}

In [258]:
tweets = pd.DataFrame(tw, columns = ['hashtags', 'tweet_id', 'text'])

In [259]:
tweets

,hashtags,tweet_id,text
0,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ..."
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #..."
2,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k..."
3,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>
4,[],0x2de201,"""Trust is not the same as faith. A friend is s..."
...,...,...,...
1867530,"[mixedfeeling, butimTHATperson]",0x316b80,When you buy the last 2 tickets remaining for ...
1867531,[],0x29d0cb,I swear all this hard work gone pay off one da...
1867532,[],0x2a6a4f,@Parcel2Go no card left when I wasn't in so I ...
1867533,[],0x24faed,"Ah, corporate life, where you can date <LH> us..."


In [50]:
identity = pd.read_csv("data_identification.csv") 

In [53]:
identity

,tweet_id,identification
0,0x28cc61,test
1,0x29e452,train
2,0x2b3819,train
3,0x2db41f,test
4,0x2a2acc,train
...,...,...
1867530,0x227e25,train
1867531,0x293813,train
1867532,0x1e1a7e,train
1867533,0x2156a5,train


In [260]:
tweets = tweets.merge(identity, left_on = 'tweet_id', right_on = 'tweet_id' )

In [261]:
tweets

,hashtags,tweet_id,text,identification
0,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...",train
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train
2,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k...",test
3,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train
4,[],0x2de201,"""Trust is not the same as faith. A friend is s...",test
...,...,...,...,...
1867530,"[mixedfeeling, butimTHATperson]",0x316b80,When you buy the last 2 tickets remaining for ...,test
1867531,[],0x29d0cb,I swear all this hard work gone pay off one da...,test
1867532,[],0x2a6a4f,@Parcel2Go no card left when I wasn't in so I ...,test
1867533,[],0x24faed,"Ah, corporate life, where you can date <LH> us...",train


In [115]:
def Upper(sent):
    return len(re.findall(r'[A-Z]',sent))
def Question(sent):
    return len(re.findall(r'\?',sent))
def Exclam(sent):
    return len(re.findall(r'\!',sent))

In [262]:
tweets['upper'] = tweets['text'].apply(lambda x: Upper(x))
tweets['question'] = tweets['text'].apply(lambda x: Question(x))
tweets['exclam'] = tweets['text'].apply(lambda x: Exclam(x))


In [263]:
tweets

,hashtags,tweet_id,text,identification,upper,question,exclam
0,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...",train,5,0,0
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train,12,0,0
2,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k...",test,8,0,0
3,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train,8,0,0
4,[],0x2de201,"""Trust is not the same as faith. A friend is s...",test,9,0,0
...,...,...,...,...,...,...,...
1867530,"[mixedfeeling, butimTHATperson]",0x316b80,When you buy the last 2 tickets remaining for ...,test,7,0,0
1867531,[],0x29d0cb,I swear all this hard work gone pay off one da...,test,3,0,0
1867532,[],0x2a6a4f,@Parcel2Go no card left when I wasn't in so I ...,test,6,0,0
1867533,[],0x24faed,"Ah, corporate life, where you can date <LH> us...",train,3,0,0


In [264]:
def process(sent):
    sent = re.sub('<LH>', '', sent)
    sent = re.sub('@[A-Z0-9a-z_:]+', '', sent)
    sent = re.sub('\#', '', sent)
#    tokens = []
#    for d in nltk.sent_tokenize(text, language='english'):
#        for word in nltk.word_tokenize(d, language='english'):
#            # filters here
#            tokens.append(word)
    return sent

In [265]:
tweets['cleaned'] =  tweets['text'].apply(lambda x: process(x))

In [266]:
tweets

,hashtags,tweet_id,text,identification,upper,question,exclam,cleaned
0,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...",train,5,0,0,"People who post ""add me on Snapchat"" must be d..."
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train,12,0,0,"As we see, Trump is dangerous to freepress ar..."
2,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k...",test,8,0,0,"Confident of your obedience, I write to you, k..."
3,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train,8,0,0,Now ISSA is stalking Tasha 😂😂😂
4,[],0x2de201,"""Trust is not the same as faith. A friend is s...",test,9,0,0,"""Trust is not the same as faith. A friend is s..."
...,...,...,...,...,...,...,...,...
1867530,"[mixedfeeling, butimTHATperson]",0x316b80,When you buy the last 2 tickets remaining for ...,test,7,0,0,When you buy the last 2 tickets remaining for ...
1867531,[],0x29d0cb,I swear all this hard work gone pay off one da...,test,3,0,0,I swear all this hard work gone pay off one da...
1867532,[],0x2a6a4f,@Parcel2Go no card left when I wasn't in so I ...,test,6,0,0,no card left when I wasn't in so I have no id...
1867533,[],0x24faed,"Ah, corporate life, where you can date <LH> us...",train,3,0,0,"Ah, corporate life, where you can date using ..."


In [206]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

stopWords = list(stopwords.words('english'))


In [267]:
train_df = tweets[tweets['identification'] == 'train']
test_df = tweets[tweets['identification'] == 'test']

In [179]:
emotions = pd.read_csv("emotion.csv") 


In [64]:
len(emotions)

1455563

In [65]:
emotions

,tweet_id,emotion
0,0x3140b1,sadness
1,0x368b73,disgust
2,0x296183,anticipation
3,0x2bd6e1,joy
4,0x2ee1dd,anticipation
...,...,...
1455558,0x38dba0,joy
1455559,0x300ea2,joy
1455560,0x360b99,fear
1455561,0x22eecf,joy


In [268]:
train_df = train_df.merge(emotions, left_on = 'tweet_id', right_on = 'tweet_id' )

In [269]:
train_df

,hashtags,tweet_id,text,identification,upper,question,exclam,cleaned,emotion
0,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...",train,5,0,0,"People who post ""add me on Snapchat"" must be d...",anticipation
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train,12,0,0,"As we see, Trump is dangerous to freepress ar...",sadness
2,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train,8,0,0,Now ISSA is stalking Tasha 😂😂😂,fear
3,"[authentic, LaughOutLoud]",0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,train,23,0,3,Thx for the BEST TIME tonight. What stories!...,joy
4,[],0x2c91a8,Still waiting on those supplies Liscus. <LH>,train,4,0,0,Still waiting on those supplies Liscus.,anticipation
...,...,...,...,...,...,...,...,...,...
1455558,"[NoWonder, Happy]",0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,train,22,0,5,I'm SO HAPPY!!! NoWonder the name of this show...,joy
1455559,[],0x38959e,In every circumtance I'd like to be thankful t...,train,7,0,0,In every circumtance I'd like to be thankful t...,joy
1455560,[blessyou],0x2cbca6,there's currently two girls walking around the...,train,2,0,0,there's currently two girls walking around the...,joy
1455561,[],0x24faed,"Ah, corporate life, where you can date <LH> us...",train,3,0,0,"Ah, corporate life, where you can date using ...",joy


In [270]:
test_df

,hashtags,tweet_id,text,identification,upper,question,exclam,cleaned
2,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k...",test,8,0,0,"Confident of your obedience, I write to you, k..."
4,[],0x2de201,"""Trust is not the same as faith. A friend is s...",test,9,0,0,"""Trust is not the same as faith. A friend is s..."
9,"[materialism, money, possessions]",0x218443,When do you have enough ? When are you satisfi...,test,5,3,0,When do you have enough ? When are you satisfi...
30,"[GodsPlan, GodsWork]",0x2939d5,"God woke you up, now chase the day #GodsPlan #...",test,7,0,0,"God woke you up, now chase the day GodsPlan Go..."
33,[],0x26289a,"In these tough times, who do YOU turn to as yo...",test,6,1,0,"In these tough times, who do YOU turn to as yo..."
...,...,...,...,...,...,...,...,...
1867525,[],0x2913b4,"""For this is the message that ye heard from th...",test,8,0,0,"""For this is the message that ye heard from th..."
1867529,[],0x2a980e,"""There is a lad here, which hath five barley l...",test,4,1,0,"""There is a lad here, which hath five barley l..."
1867530,"[mixedfeeling, butimTHATperson]",0x316b80,When you buy the last 2 tickets remaining for ...,test,7,0,0,When you buy the last 2 tickets remaining for ...
1867531,[],0x29d0cb,I swear all this hard work gone pay off one da...,test,3,0,0,I swear all this hard work gone pay off one da...


In [271]:
## save to pickle file
train_df.to_pickle("train_df.pkl") 
test_df.to_pickle("test_df.pkl")

In [75]:
train_df.groupby(['emotion']).count()['text']

emotion
anger            39867
anticipation    248935
disgust         139101
fear             63999
joy             516017
sadness         193437
surprise         48729
trust           205478
Name: text, dtype: int64

In [186]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features = 20000, tokenizer=nltk.word_tokenize)
vectorizer.fit(train_df['cleaned'])


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=20000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function word_tokenize at 0x7fab52938268>,
                use_idf=True, vocabulary=None)

In [187]:
X_train = vectorizer.transform(train_df['cleaned'])
Y_train = train_df['emotion']
X_test = vectorizer.transform(test_df['cleaned'])

#print('y_test.shape: ', y_test.shape)

X_train.shape:  (1455563, 20000)


NameError: name 'y_train' is not defined

In [188]:
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', Y_train.shape)
print('X_test.shape: ', X_test.shape)

X_train.shape:  (1455563, 20000)
y_train.shape:  (1455563,)
X_test.shape:  (411972, 20000)


In [189]:
from sklearn.naive_bayes import MultinomialNB
NB_model = MultinomialNB()
NB_model = NB_model.fit(X_train, Y_train)


In [190]:
Y_train_pred = NB_model.predict(X_train)
Y_test_pred = NB_model.predict(X_test)

In [192]:
from sklearn.metrics import accuracy_score

acc_train = accuracy_score(y_true=Y_train, y_pred=Y_train_pred)

print('training accuracy: {}'.format(round(acc_train, 2)))


training accuracy: 0.52


In [194]:
test_df['NB'] = Y_test_pred

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [195]:
test_df

,hashtags,tweet_id,text,identification,upper,question,exclam,cleaned,tokens,NB
2,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k...",test,8,0,0,"Confident of your obedience, I write to you, k...","[confident, obedience, ,, write, ,, knowing, e...",anticipation
4,[],0x2de201,"""Trust is not the same as faith. A friend is s...",test,9,0,0,"""Trust is not the same as faith. A friend is s...","[``, trust, faith, ., friend, someone, trust, ...",anticipation
9,"[materialism, money, possessions]",0x218443,When do you have enough ? When are you satisfi...,test,5,3,0,When do you have enough ? When are you satisfi...,"[enough, ?, satisfied, ?, goal, really, money,...",joy
30,"[GodsPlan, GodsWork]",0x2939d5,"God woke you up, now chase the day #GodsPlan #...",test,7,0,0,"God woke you up, now chase the day GodsPlan Go...","[god, woke, ,, chase, day, godsplan, godswork]",joy
33,[],0x26289a,"In these tough times, who do YOU turn to as yo...",test,6,1,0,"In these tough times, who do YOU turn to as yo...","[tough, times, ,, turn, symbol, hope, ?]",anticipation
...,...,...,...,...,...,...,...,...,...,...
1867525,[],0x2913b4,"""For this is the message that ye heard from th...",test,8,0,0,"""For this is the message that ye heard from th...","[``, message, ye, heard, beginning, ,, love, o...",anticipation
1867529,[],0x2a980e,"""There is a lad here, which hath five barley l...",test,4,1,0,"""There is a lad here, which hath five barley l...","[``, lad, ,, hath, five, barley, loaves, ,, tw...",anticipation
1867530,"[mixedfeeling, butimTHATperson]",0x316b80,When you buy the last 2 tickets remaining for ...,test,7,0,0,When you buy the last 2 tickets remaining for ...,"[buy, last, 2, tickets, remaining, show, sell,...",joy
1867531,[],0x29d0cb,I swear all this hard work gone pay off one da...,test,3,0,0,I swear all this hard work gone pay off one da...,"[swear, hard, work, gone, pay, one, day😈💰💸]",joy


In [197]:
test_df.to_csv (r'NB.csv', index = None, header=True, columns = ['tweet_id', 'NB']) 

In [219]:
new_vectorizer = TfidfVectorizer(analyzer = 'word', max_features = 20000, 
                                 tokenizer=nltk.word_tokenize, stop_words = stopWords, dtype = '')
new_vectorizer.fit(train_df['cleaned'])




/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=20000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...],
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function word_tokenize at 0x7fab52938268>,
                use_idf=True, vocabulary=None)

In [272]:
X_train = vectorizer.transform(train_df['cleaned'])
Y_train = train_df['emotion']
X_test = vectorizer.transform(test_df['cleaned'])

In [221]:
from sklearn.naive_bayes import MultinomialNB
NB_model = MultinomialNB()
NB_model = NB_model.fit(X_train, Y_train)
Y_train_pred = NB_model.predict(X_train)
Y_test_pred = NB_model.predict(X_test)

In [222]:
acc_train = accuracy_score(y_true=Y_train, y_pred=Y_train_pred)

print('training accuracy: {}'.format(round(acc_train, 2)))

training accuracy: 0.52


In [223]:
test_df['New_NB'] = Y_test_pred

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [224]:
test_df.to_csv (r'NB2.csv', index = None, header=True, columns = ['tweet_id', 'NB'], index_label = ['id', 'emotion']) 

In [273]:
import tensorflow as tf
import tensorflow.keras as keras

## check dimension is a good habbit 
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', Y_train.shape)
print('X_test.shape: ', X_test.shape)


X_train.shape:  (1455563, 20000)
y_train.shape:  (1455563,)
X_test.shape:  (411972, 20000)


In [283]:
type(X_train)

scipy.sparse.csr.csr_matrix

In [274]:
## deal with label (string -> one-hot)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(Y_train)


def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

Y_train_one = label_encode(label_encoder, Y_train)



In [275]:
input_shape = X_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

input_shape:  20000
output_shape:  8


In [276]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.layers import ReLU, Softmax


# input layer
model_input = Input(shape=(input_shape, ))  # 500
X = model_input

# 1st hidden layer
X_W1 = Dense(units=200)(X)  # 64
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=64)(H1)  # 64
H2 = ReLU()(H1_W2)

# output layer
H2_W3 = Dense(units=output_shape)(H2)  # 8
H3 = Softmax()(H2_W3)

model_output = H3

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 20000)]           0         
_________________________________________________________________
dense_3 (Dense)              (None, 200)               4000200   
_________________________________________________________________
re_lu_2 (ReLU)               (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                12864     
_________________________________________________________________
re_lu_3 (ReLU)               (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 520       
_________________________________________________________________
softmax_1 (Softmax)          (None, 8)                 0   

In [341]:
from tensorflow.keras.callbacks import CSVLogger

csv_logger = CSVLogger('logs/training_log.csv')


model.fit_generator(generator=training_batch_generator,
                   steps_per_epoch = int(X_split_train.shape[0] // 64),
                   epochs = 10,
                   verbose = 1,
                   workers = 4,
                   validation_data = validation_batch_generator,
                   validation_steps = int(X_split_val.shape[0] // 64))
print('training finish')


Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/utils/data_utils.py", line 742, in _run
    sequence = list(range(len(self.sequence)))
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/utils/data_utils.py", line 396, in __len__
    raise NotImplementedError
NotImplementedError



Epoch 1/10


KeyboardInterrupt: 

In [305]:
from sklearn.model_selection import train_test_split
X_split_train, X_split_val, y_split_train, y_split_val = train_test_split(
    X_train, Y_train_one, test_size=0.2, random_state=1)


In [342]:
scipy.sparse.save_npz('X_split_train', X_split_train)

In [343]:
scipy.sparse.save_npz('X_split_val', X_split_val)

In [ ]:
import numpy as np
from tensorflow import keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=64, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = np.load('data/' + ID + '.npy')

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

'data/model1/train/data0.npz'

In [304]:
#sparse_matrix = scipy.sparse.load_npz('/tmp/sparse_matrix.npz')


(1455563, 8)

In [337]:
training_batch_generator = My_Custom_Generator(X_train_filenames, y_split_train)
validation_batch_generator = My_Custom_Generator(X_val_filenames, y_split_val)

(291113, 20000)

/home/nlplab/kevintuan/2019_Data_Mining/HW2/DM19-Lab2-Homework


In [339]:
X_split_train.shape[0]

1164450